## End-to-end demonstration of RAG development
- Includes development of Vector store that is saved as storage
- Develops two query engine, used to process queries based on FT+RAG and Baseline + RAG

It is recommended to skip the 'building vector store with documents' section and move directly to load vector store.

If it is wished build the vector store it is neccessary to retrieve the following files and move to datasets
    
    "datasets/headphones_1of2.json",
    
    "datasets/headphones_2of2.json",
    
    "datasets/speakers_1of2.json",
    
    "datasets/all_models_appended.csv",
    
    "datasets/Question-for-second-evaluaton.xlsx"

#### For testing individual queries allows to run single queries based on the query engine
#### Evaluation of dataset section automatically develops an excel file with question evaluations


In [33]:
import getpass
import os
from llama_index.core import Document
import faiss
import openai
import pandas as pd
from llama_index.llms.openai import OpenAI
import json

from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display
API_KEY = 'insert api key here'
openai.api_key = API_KEY


## building vector store with documents

In [34]:
#### FULL DATA LOADERS

# dimensions of text-ada-embedding-002
d = 1536

faiss_index = faiss.IndexFlatL2(d)

#Writing 'sections' dictionary to a JSON file
with open("datasets/headphones_1of2.json", 'r', encoding='utf-8') as f:
    faqs_1 = json.load(f)
with open("datasets/headphones_2of2.json", 'r', encoding='utf-8') as f:
    faqs_2 = json.load(f)
with open("datasets/speakers_1of2.json", 'r', encoding='utf-8') as f:
    faqs_3 = json.load(f)


df = pd.read_csv('datasets/all_models_appended.csv', delimiter='~')

dff = df[df['model'] == 'gpt3.5e']
dff['full_qa'] = dff['llm_question'] + ' ' + dff['llm_answer']

faqs_beoworld = dff['full_qa'].to_list()


/var/folders/vl/zjh9b6rs2hb_pv6wtvvf6gbw0000gn/T/ipykernel_65625/3022546896.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['full_qa'] = dff['llm_question'] + ' ' + dff['llm_answer']


In [43]:
i = 0
i += len(faqs_1)
i += len(faqs_2)
i += len(faqs_3)
print(i)

1796


In [17]:
documents = []
# Handling FAQs
for faqs in [faqs_1,faqs_2,faqs_3]:
    for q,v in faqs.items():
        doc = Document(text=q+' '+v)
        documents.append(doc)

for d in faqs_beoworld:
    doc = Document(text=d)
    documents.append(doc)

In [18]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, 
)

In [19]:
# save index to disk
index.storage_context.persist()

## Load vector store

In [2]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage/"
)
index = load_index_from_storage(storage_context=storage_context)

In [20]:
len(index.index_struct.nodes_dict)


3393

## For testing individual queries

In [30]:
# query engine vanilla gpt35
llm_gpt35 = OpenAI(model='gpt-3.5-turbo-0125', api_key=API_KEY, temperature=1)
query_engine_gpt35 = index.as_query_engine(similarity_top_k=2, llm=llm_gpt35)

# query engine finetuned gpt35
llm_ft = OpenAI(model="ft:gpt-3.5-turbo-0125:personal:master:9Bfq5w4r", api_key=API_KEY, temperature=1)
query_engine_ft = index.as_query_engine(similarity_top_k=2, llm=llm_ft)

def rag_query(query_engine, query):
    response = query_engine.query(query)
    context = [node.text for node in response.source_nodes]
    return response.response, context

query = "I cant connect to bluetooh on my BeoSound A5 "
response, context = rag_query(query_engine_gpt35, query)
print(response)
print(context)

Press and hold the Bluetooth button for two seconds on your BeoSound A5 to initiate the Bluetooth connection. Once the light indicator starts flashing blue, your BeoSound A5 is ready to pair with your device. Remember to open the Bluetooth settings on your device to complete the pairing process.
['Beosound\u202fA5\u202fBluetooth\u202fconnection  Switch on\u202fBeosound\u202fA5. Press and hold\u202fthe Bluetooth\u202fbutton for\u202f\u202ftwo\u202fseconds.\u202fThe\u202flight indicator will start flashing blue, and your\u202fBeosound\u202fA5 is ready to connect to your device (phone, computer, tablet, etc.). \n--- \n---Open the Bluetooth\u202fsettings on your device and pair with the speaker.', 'Beoplay P2 Bluetooth connection']


In [31]:
context[0]

'Beosound\u202fA5\u202fBluetooth\u202fconnection  Switch on\u202fBeosound\u202fA5. Press and hold\u202fthe Bluetooth\u202fbutton for\u202f\u202ftwo\u202fseconds.\u202fThe\u202flight indicator will start flashing blue, and your\u202fBeosound\u202fA5 is ready to connect to your device (phone, computer, tablet, etc.). \n--- \n---Open the Bluetooth\u202fsettings on your device and pair with the speaker.'

In [23]:
query_engine_gpt35.get_prompts()

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x130472840>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYS

## Evaluation of dataset


In [71]:
excel_path = "datasets/Question-for-second-evaluaton.xlsx"

def rag_query(row, query_engine):
    query = row['Question']
    response = query_engine.query(query)  # Make sure the query_engine object has a query method
    context = [node.text for node in response.source_nodes]
    return pd.Series([response.response, context])

# query engine baseline gpt35
llm_gpt35 = OpenAI(model='gpt-3.5-turbo-0125', api_key=API_KEY, temperature=1)
query_engine_gpt35 = index.as_query_engine(similarity_top_k=2, llm=llm_gpt35)

# query engine finetuned gpt35
llm_emil = OpenAI(model="ft:gpt-3.5-turbo-0125:personal:master:9Bfq5w4r", api_key=API_KEY, temperature=1)
query_engine_emil = index.as_query_engine(similarity_top_k=2, llm=llm_emil)

# Load your DataFrame
df_eval = pd.read_excel(excel_path)

for name in ['rag', 'ft_rag']:
    if name == 'rag':
        query_engine = query_engine_gpt35
    if name == 'ft_rag':
        query_egnine = query_engine_emil
    # Apply the rag_query function across the DataFrame
    df_eval[['Answer', 'context']] = df_eval.apply(lambda row: rag_query(row, query_engine), axis=1)
    df_eval.to_excel(f'eval_{name}.xlsx')